https://www.kaggle.com/anonamename/fork-cassava-emsemble-v2-for-train

In [1]:
!pwd
import sys

%reload_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext lab_black
sys.executable

/c/Users/81908/jupyter_notebook/pytorch_lightning_work/kaggle_Cassava/notebook/check_oof/cassava-emsemble-v2_tta_oof


'C:\\Users\\81908\\Anaconda3\\envs\\lightning\\python.exe'

In [2]:
#!pip install "../input/pytorchlightning110/pytorch_lightning-1.1.0-py3-none-any.whl"
#
# import os
# import sys
# sys.path.append("../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master")
# sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')

In [3]:
import os, sys

if os.getcwd() in ["/kaggle/working", "/content"]:
    if not os.path.exists("./kaggle_Cassava"):
        !git clone https://github.com/riron1206/kaggle_Cassava.git
    sys.path.append("./kaggle_Cassava/code")
else:
    sys.path.append(r"C:\Users\81908\MyGitHub\kaggle_Cassava\code")
    # sys.path.append(r"C:\Users\shingo\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\kaggle_Cassava\code")

from microsoftvision_resnet50 import MicrosoftVisionResnet50

In [5]:
import os
import gc
import cv2
import glob
import json
import yaml
import pickle
import random
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, log_loss
from tqdm.notebook import tqdm
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import timm
import pytorch_lightning as pl
from efficientnet_pytorch import EfficientNet
from deit_models import deit_base_patch16_224
import warnings

warnings.filterwarnings("ignore")

In [6]:
print("timm version:", timm.__version__)
print("pytorch_lightning version:", pl.__version__)

timm version: 0.3.2
pytorch_lightning version: 1.1.0


In [7]:
DEBUG = False
# DEBUG = True

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)

device: cuda


In [9]:
def set_seed(seed: int = 0) -> None:
    np.random.seed(seed)
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

In [10]:
class CassavaLite(pl.LightningModule):
    def __init__(self, source="timm", arch="seresnext50_32x4d"):
        super().__init__()

        if source == "efficientnet-pytorch":
            self.net = EfficientNet.from_name(arch)
            self.net._fc = nn.Linear(self.net._fc.in_features, n_classes)

        elif source == "timm":
            self.net = timm.create_model(arch, pretrained=None)
            if "eff" in arch:
                self.net.classifier = nn.Linear(
                    self.net.classifier.in_features, n_classes
                )
            elif "rexnet" in arch:
                self.net.head.fc = nn.Linear(self.net.head.fc.in_features, n_classes)
            elif "vit" in arch:
                self.net.head = nn.Linear(self.net.head.in_features, n_classes)
            else:
                self.net.fc = nn.Linear(self.net.fc.in_features, n_classes)

    def forward(self, x):
        out = self.net(x)
        return out


class CassavaLite2(pl.LightningModule):
    def __init__(self, arch="vit_base_patch16_224"):
        super().__init__()

        self.net = timm.create_model(arch, pretrained=None)

        if "eff" in arch:
            self.net.classifier = nn.Linear(self.net.classifier.in_features, n_classes)
            self.feat_net = nn.Sequential(*list(self.net.children())[:-2])
        elif "rexnet" in arch:
            self.net.head.fc = nn.Linear(self.net.head.fc.in_features, n_classes)
            self.feat_net = nn.Sequential(*list(self.net.children())[:-1])
        elif "vit" in arch:
            self.net.head = nn.Linear(self.net.head.in_features, n_classes)
            self.feat_net = nn.Sequential(*list(self.net.children())[:-1])
        else:
            self.net.fc = nn.Linear(self.net.fc.in_features, n_classes)
            self.feat_net = nn.Sequential(
                *list(self.net.children())[:-2]
            )  # global_poolとfc層除く

    def forward(self, x):
        out = self.net(x)
        return out


class CassavaLite3(pl.LightningModule):
    def __init__(self, CFG):
        super().__init__()

        self.net = timm.create_model(CFG.arch, pretrained=None)

        if "eff" in CFG.arch:
            self.net.classifier = nn.Linear(
                self.net.classifier.in_features, CFG.n_classes
            )
            if CFG.gem_p > 0.0:
                self.net = GeMNet(
                    list(self.net.children())[:-2],
                    GeM(p=CFG.gem_p),
                    self.net.classifier.in_features,
                    CFG.n_classes,
                )
            self.feat_net = nn.Sequential(*list(self.net.children())[:-2])

        elif "rexnet" in CFG.arch:
            self.net.head.fc = nn.Linear(self.net.head.fc.in_features, CFG.n_classes)
            if CFG.gem_p > 0.0:
                self.net = GeMNet(
                    list(self.net.children())[:-1],
                    GeM(p=CFG.gem_p),
                    self.net.head.fc.in_features,
                    CFG.n_classes,
                )
            self.feat_net = nn.Sequential(*list(self.net.children())[:-1])

        elif "vit" in CFG.arch:
            self.net.head = nn.Linear(self.net.head.in_features, CFG.n_classes)
            self.feat_net = nn.Sequential(*list(self.net.children())[:-1])

        else:
            self.net.fc = nn.Linear(self.net.fc.in_features, CFG.n_classes)
            if CFG.gem_p > 0.0:
                self.net = GeMNet(
                    list(self.net.children())[:-2],
                    GeM(p=CFG.gem_p),
                    self.net.fc.in_features,
                    CFG.n_classes,
                )
            self.feat_net = nn.Sequential(
                *list(self.net.children())[:-2]
            )  # global_poolとfc層除く

    def forward(self, x):
        out = self.net(x)
        return out


class CassavaLite4(pl.LightningModule):
    def __init__(self, CFG):
        super().__init__()

        if "microsoft" in CFG.arch:
            self.net = MicrosoftVisionResnet50(
                pretrained=False, n_classes=CFG.n_classes
            )
            self.feat_net = nn.Sequential(*list(self.net.children())[:-1])

        else:
            self.net = timm.create_model(CFG.arch, pretrained=None)

            if "eff" in CFG.arch:
                self.net.classifier = nn.Linear(
                    self.net.classifier.in_features, CFG.n_classes
                )
                self.feat_net = nn.Sequential(*list(self.net.children())[:-2])

            elif "rexnet" in CFG.arch:
                self.net.head.fc = nn.Linear(
                    self.net.head.fc.in_features, CFG.n_classes
                )
                self.feat_net = nn.Sequential(*list(self.net.children())[:-1])

            elif "vit" in CFG.arch:
                self.net.head = nn.Linear(self.net.head.in_features, CFG.n_classes)
                self.feat_net = nn.Sequential(*list(self.net.children())[:-1])

            elif "deit" in CFG.arch:
                self.net = deit_base_patch16_224(
                    pretrained=False
                )  # pretrained=True にすると失敗する。。。
                self.net.head = nn.Linear(self.net.head.in_features, CFG.n_classes)
                self.feat_net = nn.Sequential(*list(self.net.children())[:-1])

            else:
                self.net.fc = nn.Linear(self.net.fc.in_features, CFG.n_classes)
                self.feat_net = nn.Sequential(
                    *list(self.net.children())[:-2]
                )  # global_poolとfc層除く

    def forward(self, x):
        out = self.net(x)
        return out

In [11]:
class CassavaDataset(Dataset):
    def __init__(self, df: pd.DataFrame, transformgroups):
        # def __init__(self, df: pd.DataFrame, imfolder: str, transformgroups):
        self.df = df
        # self.imfolder = imfolder
        self.transformgroups = transformgroups

    def __getitem__(self, index):
        im_path = self.df.iloc[index]["file_path"]
        # im_path = os.path.join(self.imfolder, self.df.iloc[index]["image_id"])
        x = cv2.imread(im_path, cv2.IMREAD_COLOR)
        x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
        y = self.df.iloc[index]["label"]
        ret = {}
        for i, g in enumerate(self.transformgroups):
            v = None
            for t in g:
                r = t(image=x)["image"].unsqueeze(0)
                v = r if v is None else torch.cat([v, r], axis=0)
            ret[f"x{i}"] = v.float()
        return ret, y

    def __len__(self):
        return len(self.df)

In [12]:
def get_models(model_paths, lite_type, CFG=None):
    models = []
    for model_path in model_paths:
        if lite_type == "CassavaLite":
            model = CassavaLite().load_from_checkpoint(model_path).to(device)
        elif lite_type == "CassavaLite2":
            model = CassavaLite2().load_from_checkpoint(model_path).to(device)
        elif lite_type == "CassavaLite3":
            model = (
                CassavaLite3(CFG).load_from_checkpoint(model_path, CFG=CFG).to(device)
            )
        elif lite_type == "CassavaLite4":
            model = (
                CassavaLite4(CFG).load_from_checkpoint(model_path, CFG=CFG).to(device)
            )
        model.eval()
        models.append(model)
        print(f"{model_path} is loaded")
    return models


def get_dataloader(img_size_desc, test_df):
    transforms = []
    for item in img_size_desc:
        # model = item[0]
        tta_nums = item[1]  # ttaの回数
        img_size = item[2]
        if img_size < 500:
            t = A.Compose(
                [
                    A.RandomResizedCrop(
                        img_size,
                        img_size,
                        #    p=0.5
                    ),
                    A.Normalize(
                        mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225],
                        max_pixel_value=255.0,
                        p=1.0,
                    ),
                    ToTensorV2(p=1.0),
                ],
                p=1.0,
            )
        else:
            t = A.Compose(
                [
                    A.CenterCrop(
                        img_size,
                        img_size,
                        #    p=0.5
                    ),
                    A.Normalize(
                        mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225],
                        max_pixel_value=255.0,
                        p=1.0,
                    ),
                    ToTensorV2(p=1.0),
                ],
                p=1.0,
            )
        transforms.append([t for _ in range(tta_nums)])
    # test_dataset = CassavaDataset(test_df, img_dir, transforms)
    test_dataset = CassavaDataset(test_df, transforms)
    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        shuffle=False,
        drop_last=False,
    )
    return test_loader

# main

In [13]:
n_classes = 5
# num_workers = 4
num_workers = 0

path = r"C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\input\cassava-leaf-disease-classification"
img_dir = f"{path}/train_images/"
df = pd.read_csv(f"{path}/train.csv")
df["file_path"] = f"{img_dir}/" + df["image_id"]
if DEBUG:
    df = df.iloc[:20, :]

with open(f"{path}/label_num_to_disease_map.json", "r") as f:
    name_mapping = json.load(f)
name_mapping = {int(k): v for k, v in name_mapping.items()}

In [14]:
n_tta = 5
n_tta_many = 8
# batch_size = 16
batch_size = 1
test_preds_list = []

In [15]:
def randomTTA(x):
    if np.random.rand() > 0.5:
        x = torch.flip(x, [-1])
    if np.random.rand() > 0.5:
        x = torch.flip(x, [-2])
    return x

In [16]:
class Config:
    def __init__(self, **kwargs):
        self.seeds = kwargs["seeds"]
        self.n_classes = n_classes
        self.arch = kwargs["arch"]
        self.gem_p = kwargs["gem_p"]
        self.height = kwargs["height"]


def GetYaml(yamlpath):
    with open(yamlpath, "r") as f:
        cdict = yaml.load(f)
    cfg = Config(**cdict)
    return cfg

In [17]:
def check_train_oof(models, random_state, df):
    mdlgrps = models[0][0]
    numtta = models[0][1]
    n_splits = len(mdlgrps)

    set_seed(seed=random_state)
    Y_pred = pd.DataFrame(np.zeros((df.shape[0], 5)), columns=name_mapping.values())
    Y_hardvote = pd.DataFrame(np.zeros((df.shape[0], 1)))
    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    for k, (train_idx, valid_idx) in tqdm(enumerate(cv.split(df, df["label"]))):
        val_df = df.iloc[valid_idx]
        loader = get_dataloader(models, val_df)
        mdl = mdlgrps[k]
        with torch.no_grad():
            preds = []
            preds_hardvote = []
            actfn = nn.Softmax(dim=1)
            for batch in loader:
                ret, label = batch
                img = ret["x0"][0].to(device)  # numttaの枚数分同じtensor型の画像返す
                label = label.long().to(device)
                b = img.shape[0]  # numttaと同じ値になる
                x = img
                y = torch.zeros([b, 5], device=device)
                for j in range(b):  # numttaの枚数ループ
                    x[j] = randomTTA(img[j])  # ttaランダムに入れる
                z = mdl(x)
                y = y + actfn(z)
                y = y.detach().cpu().numpy()
                y_hard = max([y_i.argmax() for y_i in y])  # ttaをhardvote
                preds_hardvote += [y_hard]
                y_mean = np.mean(y, axis=0)  # ttaの予測平均化
                preds += [y_mean]
        # print(preds)
        # print(preds_hardvote)
        Y_pred.iloc[valid_idx] = np.array(preds)
        Y_hardvote.iloc[valid_idx] = np.array(preds_hardvote).reshape(
            len(preds_hardvote), 1
        )
    oof = accuracy_score(df["label"].values, Y_pred.values.argmax(1))
    print(f"oof:", round(oof, 4))
    oof_hardvote = accuracy_score(df["label"].values, Y_hardvote.values)
    print(f"oof_hardvote:", round(oof_hardvote, 4))
    return Y_pred, Y_hardvote

In [18]:
outdir = "."
os.makedirs(outdir, exist_ok=True)

In [20]:
# tf_efficientnet_b4_ns
# oof 0.894
m_dir = r"C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\notebook\cassava-bitempered-logistic-loss\20201227\kaggle_upload_tf_efficientnet_b4_ns_BiTemperedLoss"
cfg1 = GetYaml(f"{m_dir}/hparams.yaml")
mdl1 = sorted(
    glob.glob(
        f"{m_dir}/model_seed_*.ckpt"
    )
)
models1 = get_models(mdl1, "CassavaLite3", CFG=cfg1)

m_name = f"tf_efficientnet_b4_ns"
models = [(models1, n_tta, 512)]
random_state = cfg1.seeds[0]
Y_tta, Y_tta_hardvote = check_train_oof(models, random_state, df)
pickle.dump(Y_tta, open(f"{outdir}/{m_name}_tta.pkl", "wb"))
pickle.dump(Y_tta_hardvote, open(f"{outdir}/{m_name}_tta_hardvote.pkl", "wb"))
# display(Y_pred)
# display(Y_hardvote)
del models
torch.cuda.empty_cache()
gc.collect()


oof: 0.8924
oof_hardvote: 0.8925


3687

In [21]:
# byol + seresnext50_32x4d
# oof -.---
m_dir = r"C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\run_old_myPC\notebook\kaggle_upload_byol_seresnext50_32x4d_cutmix_labelsmooth_half"
cfg2 = GetYaml(f"{m_dir}/cfg.yaml")
mdl2 = sorted(
    glob.glob(
        f"{m_dir}/model_seed_*.ckpt"
    )
)
models2 = get_models(mdl2, "CassavaLite4", CFG=cfg2)

m_name = f"byol_seresnext50_32x4d_cutmix_labelsmooth_half"
models = [(models2, n_tta, cfg2.height)]
random_state = cfg2.seeds[0]
Y_tta, Y_tta_hardvote = check_train_oof(models, random_state, df)
pickle.dump(Y_tta, open(f"{outdir}/{m_name}_tta.pkl", "wb"))
pickle.dump(Y_tta_hardvote, open(f"{outdir}/{m_name}_tta_hardvote.pkl", "wb"))
# display(Y_pred)
# display(Y_hardvote)
del models
torch.cuda.empty_cache()
gc.collect()


oof: 0.8942
oof_hardvote: 0.8933


699

In [19]:
# vit_b16_224_fold10
# oof -.---
m_dir = r"C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\run_old_myPC\notebook\cassava-bitempered-logistic-loss\20210121\kaggle_upload_vit_base_patch16_224_fold3\vit_b16_224_fold10\kaggle_upload_vit_b16_224_fold10"
cfg3 = GetYaml(f"{m_dir}/cfg.yaml")
mdl3 = sorted(glob.glob(f"{m_dir}/model_seed_*.ckpt"))
models3 = get_models(mdl3, "CassavaLite4", CFG=cfg3)

m_name = f"vit_b16_224_fold10"
models = [(models3, n_tta_many, cfg3.height)]
random_state = cfg3.seeds[0]
Y_tta, Y_tta_hardvote = check_train_oof(models, random_state, df)
pickle.dump(Y_tta, open(f"{outdir}/{m_name}_tta.pkl", "wb"))
pickle.dump(Y_tta_hardvote, open(f"{outdir}/{m_name}_tta_hardvote.pkl", "wb"))
# display(Y_pred)
# display(Y_hardvote)
del models
torch.cuda.empty_cache()
gc.collect()

C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\run_old_myPC\notebook\cassava-bitempered-logistic-loss\20210121\kaggle_upload_vit_base_patch16_224_fold3\vit_b16_224_fold10\kaggle_upload_vit_b16_224_fold10\model_seed_25_fold_0.ckpt is loaded
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\run_old_myPC\notebook\cassava-bitempered-logistic-loss\20210121\kaggle_upload_vit_base_patch16_224_fold3\vit_b16_224_fold10\kaggle_upload_vit_b16_224_fold10\model_seed_25_fold_1.ckpt is loaded
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\run_old_myPC\notebook\cassava-bitempered-logistic-loss\20210121\kaggle_upload_vit_base_patch16_224_fold3\vit_b16_224_fold10\kaggle_upload_vit_b16_224_fold10\model_seed_25_fold_2.ckpt is loaded
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\run_old_myPC\notebook\cassava-bitempered-logistic-loss\20210121\kaggle_upload_vit_base_patch16_224_fold3\vit_b16_224_fold10\kaggle_upload_vit


oof: 0.8861
oof_hardvote: 0.8275


25553

In [18]:
# seresnext50_32x4d_cutmixfmix 過去コンペのデータあるので注意
# oof -.---
m_dir = r"C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\notebook\efficientnet-with-pytorch-lightning-train-infer\20201217\kaggle_upload_seresnext50_32x4d_cutmixfmix"
cfg = GetYaml(f"{m_dir}/cfg.yaml")
mdl = sorted(glob.glob(f"{m_dir}/model_seed_*.ckpt"))
models = get_models(mdl, "CassavaLite", CFG=cfg)

_df = pd.read_csv(f"{m_dir}/oof_preds_df.csv")
_df = _df.rename(columns={"target": "label"})
if DEBUG:
    _df = _df.iloc[:20, :]

m_name = f"seresnext50_32x4d_cutmixfmix"
models = [(models, n_tta, cfg.height)]
random_state = cfg.seeds[0]
Y_tta, Y_tta_hardvote = check_train_oof(models, random_state, _df)

Y_tta, Y_tta_hardvote = Y_tta.iloc[:21397], Y_tta_hardvote.iloc[:21397]

pickle.dump(Y_tta, open(f"{outdir}/{m_name}_tta.pkl", "wb"))
pickle.dump(Y_tta_hardvote, open(f"{outdir}/{m_name}_tta_hardvote.pkl", "wb"))
# display(Y_pred)
# display(Y_hardvote)
del models
torch.cuda.empty_cache()
gc.collect()

C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\notebook\efficientnet-with-pytorch-lightning-train-infer\20201217\kaggle_upload_seresnext50_32x4d_cutmixfmix\model_seed_0_fold_0.ckpt is loaded
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\notebook\efficientnet-with-pytorch-lightning-train-infer\20201217\kaggle_upload_seresnext50_32x4d_cutmixfmix\model_seed_0_fold_1.ckpt is loaded
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\notebook\efficientnet-with-pytorch-lightning-train-infer\20201217\kaggle_upload_seresnext50_32x4d_cutmixfmix\model_seed_0_fold_2.ckpt is loaded


ValueError: Requested crop size (512, 512) is larger than the image size (500, 499)

In [20]:
# vit_base_patch16_224.fit
# oof -.---
m_dir = r"C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\kaggle_datasets_dl\cassava_vit_base_patch16_224.fit"
cfg = GetYaml(f"{m_dir}/cfg.yaml")
mdl = sorted(glob.glob(f"{m_dir}/model_seed_*.ckpt"))
models = get_models(mdl, "CassavaLite2", CFG=cfg)

m_name = f"vit_base_patch16_224.fit"
models = [(models, n_tta_many, cfg.height)]
random_state = cfg.seeds[0]
Y_tta, Y_tta_hardvote = check_train_oof(models, random_state, df)
pickle.dump(Y_tta, open(f"{outdir}/{m_name}_tta.pkl", "wb"))
pickle.dump(Y_tta_hardvote, open(f"{outdir}/{m_name}_tta_hardvote.pkl", "wb"))
# display(Y_pred)
# display(Y_hardvote)
del models
torch.cuda.empty_cache()
gc.collect()

C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\kaggle_datasets_dl\cassava_vit_base_patch16_224.fit\model_seed_0_fold_0.ckpt is loaded
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\kaggle_datasets_dl\cassava_vit_base_patch16_224.fit\model_seed_0_fold_1.ckpt is loaded
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\kaggle_datasets_dl\cassava_vit_base_patch16_224.fit\model_seed_0_fold_2.ckpt is loaded
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\kaggle_datasets_dl\cassava_vit_base_patch16_224.fit\model_seed_0_fold_3.ckpt is loaded
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\kaggle_datasets_dl\cassava_vit_base_patch16_224.fit\model_seed_0_fold_4.ckpt is loaded



oof: 0.8792
oof_hardvote: 0.8203


28

In [19]:
# vit_base_patch32_384.fit
# oof -.---
m_dir = r"C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\kaggle_datasets_dl\cassava_vit_base_patch32_384.fit"
cfg = GetYaml(f"{m_dir}/cfg.yaml")
mdl = sorted(glob.glob(f"{m_dir}/model_seed_*.ckpt"))
models = get_models(mdl, "CassavaLite4", CFG=cfg)

m_name = f"vit_base_patch32_384.fit"
models = [(models, n_tta_many, cfg.height)]
random_state = cfg.seeds[0]
Y_tta, Y_tta_hardvote = check_train_oof(models, random_state, df)
pickle.dump(Y_tta, open(f"{outdir}/{m_name}_tta.pkl", "wb"))
pickle.dump(Y_tta_hardvote, open(f"{outdir}/{m_name}_tta_hardvote.pkl", "wb"))
# display(Y_pred)
# display(Y_hardvote)
del models
torch.cuda.empty_cache()
gc.collect()

C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\kaggle_datasets_dl\cassava_vit_base_patch32_384.fit\model_seed_42_fold_0.ckpt is loaded
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\kaggle_datasets_dl\cassava_vit_base_patch32_384.fit\model_seed_42_fold_1.ckpt is loaded
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\kaggle_datasets_dl\cassava_vit_base_patch32_384.fit\model_seed_42_fold_2.ckpt is loaded
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\kaggle_datasets_dl\cassava_vit_base_patch32_384.fit\model_seed_42_fold_3.ckpt is loaded
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\kaggle_datasets_dl\cassava_vit_base_patch32_384.fit\model_seed_42_fold_4.ckpt is loaded



oof: 0.8831
oof_hardvote: 0.8231


18198

In [19]:
# seresnext50_32x4d_fmix
# oof -.---
m_dir = r"C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\run_old_myPC\notebook\kaggle_upload_seresnext50_32x4d_fmix"
cfg = Config(**{"arch": "seresnext50_32x4d", "gem_p": 0, "height": 512, "seeds": [0]})
mdl = sorted(glob.glob(f"{m_dir}/model_seed_*.ckpt"))
models = get_models(mdl, "CassavaLite4", CFG=cfg)

m_name = f"seresnext50_32x4d_fmix"
models = [(models, n_tta, cfg.height)]
random_state = cfg.seeds[0]
Y_tta, Y_tta_hardvote = check_train_oof(models, random_state, df)
pickle.dump(Y_tta, open(f"{outdir}/{m_name}_tta.pkl", "wb"))
pickle.dump(Y_tta_hardvote, open(f"{outdir}/{m_name}_tta_hardvote.pkl", "wb"))
# display(Y_pred)
# display(Y_hardvote)
del models
torch.cuda.empty_cache()
gc.collect()

C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\run_old_myPC\notebook\kaggle_upload_seresnext50_32x4d_fmix\model_seed_0_fold_0.ckpt is loaded
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\run_old_myPC\notebook\kaggle_upload_seresnext50_32x4d_fmix\model_seed_0_fold_1.ckpt is loaded
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\run_old_myPC\notebook\kaggle_upload_seresnext50_32x4d_fmix\model_seed_0_fold_2.ckpt is loaded
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\run_old_myPC\notebook\kaggle_upload_seresnext50_32x4d_fmix\model_seed_0_fold_3.ckpt is loaded
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\run_old_myPC\notebook\kaggle_upload_seresnext50_32x4d_fmix\model_seed_0_fold_4.ckpt is loaded



oof: 0.8834
oof_hardvote: 0.8849


28

In [18]:
# resnest101e
# oof -.---
m_dir = r"C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\run_old_myPC\notebook\cassava-bitempered-logistic-loss\20201227\kaggle_upload_resnest101e_bi_tempered_loss"
cfg = Config(**{"arch": "resnest101e", "gem_p": 0, "height": 512, "seeds": [42]})
mdl = sorted(glob.glob(f"{m_dir}/model_seed_*.ckpt"))
models = get_models(mdl, "CassavaLite4", CFG=cfg)

m_name = f"resnest101e"
models = [(models, n_tta, cfg.height)]
random_state = cfg.seeds[0]
Y_tta, Y_tta_hardvote = check_train_oof(models, random_state, df)
pickle.dump(Y_tta, open(f"{outdir}/{m_name}_tta.pkl", "wb"))
pickle.dump(Y_tta_hardvote, open(f"{outdir}/{m_name}_tta_hardvote.pkl", "wb"))
# display(Y_pred)
# display(Y_hardvote)
del models
torch.cuda.empty_cache()
gc.collect()

C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\run_old_myPC\notebook\cassava-bitempered-logistic-loss\20201227\kaggle_upload_resnest101e_bi_tempered_loss\model_seed_42_fold_0.ckpt is loaded
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\run_old_myPC\notebook\cassava-bitempered-logistic-loss\20201227\kaggle_upload_resnest101e_bi_tempered_loss\model_seed_42_fold_1.ckpt is loaded
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\run_old_myPC\notebook\cassava-bitempered-logistic-loss\20201227\kaggle_upload_resnest101e_bi_tempered_loss\model_seed_42_fold_2.ckpt is loaded
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\run_old_myPC\notebook\cassava-bitempered-logistic-loss\20201227\kaggle_upload_resnest101e_bi_tempered_loss\model_seed_42_fold_3.ckpt is loaded
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\run_old_myPC\notebook\cassava-bitempered-logistic-loss\20201227\kaggle_upload_


oof: 0.8942
oof_hardvote: 0.892


17

In [22]:
# resnest101e_cleanlab
# oof -.---
m_dir = r"C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\notebook\cleanlab_test\20210117\kaggle_upload_resnest101e_cleanlab"
cfg = GetYaml(f"{m_dir}/cfg.yaml")
mdl = sorted(glob.glob(f"{m_dir}/model_seed_*.ckpt"))
models = get_models(mdl, "CassavaLite4", CFG=cfg)

m_name = f"resnest101e_cleanlab"
models = [(models, n_tta, cfg.height)]
random_state = cfg.seeds[0]
Y_tta, Y_tta_hardvote = check_train_oof(models, random_state, df)
pickle.dump(Y_tta, open(f"{outdir}/{m_name}_tta.pkl", "wb"))
pickle.dump(Y_tta_hardvote, open(f"{outdir}/{m_name}_tta_hardvote.pkl", "wb"))
# display(Y_pred)
# display(Y_hardvote)
del models
torch.cuda.empty_cache()
gc.collect()

C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\notebook\cleanlab_test\20210117\kaggle_upload_resnest101e_cleanlab\model_seed_0_fold_0.ckpt is loaded
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\notebook\cleanlab_test\20210117\kaggle_upload_resnest101e_cleanlab\model_seed_0_fold_1.ckpt is loaded
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\notebook\cleanlab_test\20210117\kaggle_upload_resnest101e_cleanlab\model_seed_0_fold_2.ckpt is loaded
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\notebook\cleanlab_test\20210117\kaggle_upload_resnest101e_cleanlab\model_seed_0_fold_3.ckpt is loaded
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\notebook\cleanlab_test\20210117\kaggle_upload_resnest101e_cleanlab\model_seed_0_fold_4.ckpt is loaded



oof: 0.8948
oof_hardvote: 0.8931


28

In [19]:
# resnest101e_cleanlab_noise_cutmix
# oof -.---
m_dir = r"C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\run_old_myPC\notebook\cleanlab_test\20210117\kaggle_upload_resnest101e_cleanlab_noise_cutmix"
cfg = GetYaml(f"{m_dir}/cfg.yaml")
mdl = sorted(glob.glob(f"{m_dir}/model_seed_*.ckpt"))
models = get_models(mdl, "CassavaLite4", CFG=cfg)

m_name = f"resnest101e_cleanlab_noise_cutmix"
models = [(models, n_tta, cfg.height)]
random_state = cfg.seeds[0]
Y_tta, Y_tta_hardvote = check_train_oof(models, random_state, df)
pickle.dump(Y_tta, open(f"{outdir}/{m_name}_tta.pkl", "wb"))
pickle.dump(Y_tta_hardvote, open(f"{outdir}/{m_name}_tta_hardvote.pkl", "wb"))
# display(Y_pred)
# display(Y_hardvote)
del models
torch.cuda.empty_cache()
gc.collect()

C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\run_old_myPC\notebook\cleanlab_test\20210117\kaggle_upload_resnest101e_cleanlab_noise_cutmix\model_seed_0_fold_0.ckpt is loaded
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\run_old_myPC\notebook\cleanlab_test\20210117\kaggle_upload_resnest101e_cleanlab_noise_cutmix\model_seed_0_fold_1.ckpt is loaded
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\run_old_myPC\notebook\cleanlab_test\20210117\kaggle_upload_resnest101e_cleanlab_noise_cutmix\model_seed_0_fold_2.ckpt is loaded
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\run_old_myPC\notebook\cleanlab_test\20210117\kaggle_upload_resnest101e_cleanlab_noise_cutmix\model_seed_0_fold_3.ckpt is loaded
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\run_old_myPC\notebook\cleanlab_test\20210117\kaggle_upload_resnest101e_cleanlab_noise_cutmix\model_seed_0_fold_4.ckpt is loaded



oof: 0.8949
oof_hardvote: 0.8941


28

In [21]:
# deit_base_patch16_224
# oof -.---
m_dir = r"C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\gcp\jupyter\working\20210112\kaggle_upload_deit-base-patch16-224"
cfg = GetYaml(f"{m_dir}/cfg.yaml")
mdl = sorted(glob.glob(f"{m_dir}/model_seed_*.ckpt"))
models = get_models(mdl, "CassavaLite4", CFG=cfg)

m_name = f"deit_base_patch16_224"
models = [(models, n_tta_many, cfg.height)]
random_state = cfg.seeds[0]
Y_tta, Y_tta_hardvote = check_train_oof(models, random_state, df)
pickle.dump(Y_tta, open(f"{outdir}/{m_name}_tta.pkl", "wb"))
pickle.dump(Y_tta_hardvote, open(f"{outdir}/{m_name}_tta_hardvote.pkl", "wb"))
# display(Y_pred)
# display(Y_hardvote)
del models
torch.cuda.empty_cache()
gc.collect()

C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\gcp\jupyter\working\20210112\kaggle_upload_deit-base-patch16-224\model_seed_0_fold_0.ckpt is loaded
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\gcp\jupyter\working\20210112\kaggle_upload_deit-base-patch16-224\model_seed_0_fold_1.ckpt is loaded
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\gcp\jupyter\working\20210112\kaggle_upload_deit-base-patch16-224\model_seed_0_fold_2.ckpt is loaded
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\gcp\jupyter\working\20210112\kaggle_upload_deit-base-patch16-224\model_seed_0_fold_3.ckpt is loaded
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\gcp\jupyter\working\20210112\kaggle_upload_deit-base-patch16-224\model_seed_0_fold_4.ckpt is loaded



oof: 0.8853
oof_hardvote: 0.8325


28

In [20]:
# tf_efficientnet_b4_ns_fold3
# oof -.---
m_dir = r"C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\notebook\cassava-bitempered-logistic-loss\20210121\kaggle_upload_tf_efficientnet_b4_ns_fold3"
cfg = GetYaml(f"{m_dir}/cfg.yaml")
mdl = sorted(glob.glob(f"{m_dir}/model_seed_*.ckpt"))
models = get_models(mdl, "CassavaLite4", CFG=cfg)

m_name = f"tf_efficientnet_b4_ns_fold3"
models = [(models, n_tta, cfg.height)]
random_state = cfg.seeds[0]
Y_tta, Y_tta_hardvote = check_train_oof(models, random_state, df)
pickle.dump(Y_tta, open(f"{outdir}/{m_name}_tta.pkl", "wb"))
pickle.dump(Y_tta_hardvote, open(f"{outdir}/{m_name}_tta_hardvote.pkl", "wb"))
# display(Y_pred)
# display(Y_hardvote)
del models
torch.cuda.empty_cache()
gc.collect()

C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\notebook\cassava-bitempered-logistic-loss\20210121\kaggle_upload_tf_efficientnet_b4_ns_fold3\model_seed_159_fold_0.ckpt is loaded
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\notebook\cassava-bitempered-logistic-loss\20210121\kaggle_upload_tf_efficientnet_b4_ns_fold3\model_seed_159_fold_1.ckpt is loaded
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\notebook\cassava-bitempered-logistic-loss\20210121\kaggle_upload_tf_efficientnet_b4_ns_fold3\model_seed_159_fold_2.ckpt is loaded



oof: 0.8925
oof_hardvote: 0.8915


28

In [19]:
# tf_efficientnet_b4_ns_fold10
# oof -.---
m_dir = r"C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\notebook\cassava-bitempered-logistic-loss\20210122\kaggle_upload_tf_efficientnet_b4_ns_fold10"
cfg = GetYaml(f"{m_dir}/cfg.yaml")
mdl = sorted(glob.glob(f"{m_dir}/model_seed_*.ckpt"))
models = get_models(mdl, "CassavaLite4", CFG=cfg)

m_name = f"tf_efficientnet_b4_ns_fold10"
models = [(models, n_tta, cfg.height)]
random_state = cfg.seeds[0]
Y_tta, Y_tta_hardvote = check_train_oof(models, random_state, df)
pickle.dump(Y_tta, open(f"{outdir}/{m_name}_tta.pkl", "wb"))
pickle.dump(Y_tta_hardvote, open(f"{outdir}/{m_name}_tta_hardvote.pkl", "wb"))
# display(Y_pred)
# display(Y_hardvote)
del models
torch.cuda.empty_cache()
gc.collect()

C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\notebook\cassava-bitempered-logistic-loss\20210122\kaggle_upload_tf_efficientnet_b4_ns_fold10\model_seed_4562_fold_0.ckpt is loaded
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\notebook\cassava-bitempered-logistic-loss\20210122\kaggle_upload_tf_efficientnet_b4_ns_fold10\model_seed_4562_fold_1.ckpt is loaded
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\notebook\cassava-bitempered-logistic-loss\20210122\kaggle_upload_tf_efficientnet_b4_ns_fold10\model_seed_4562_fold_2.ckpt is loaded
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\notebook\cassava-bitempered-logistic-loss\20210122\kaggle_upload_tf_efficientnet_b4_ns_fold10\model_seed_4562_fold_3.ckpt is loaded
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\notebook\cassava-bitempered-logistic-loss\20210122\kaggle_upload_tf_efficientnet_b4_ns_fold10\model_seed_4562_fold_4.ckpt 


oof: 0.8921
oof_hardvote: 0.8906


19

In [21]:
# tf_efficientnet_b4_ns_cleanlab_noise_cutmix_fmix_n_over
# oof -.---
m_dir = r"C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\notebook\cleanlab_test\20210125\kaggle_upload_tf_efficientnet_b4_ns_cleanlab_noise_cutmix_fmix_n_over"
cfg = GetYaml(f"{m_dir}/cfg.yaml")
mdl = sorted(glob.glob(f"{m_dir}/model_seed_*.ckpt"))
models = get_models(mdl, "CassavaLite4", CFG=cfg)

m_name = f"tf_efficientnet_b4_ns_cleanlab_noise_cutmix_fmix_n_over"
models = [(models, n_tta, cfg.height)]
random_state = cfg.seeds[0]
Y_tta, Y_tta_hardvote = check_train_oof(models, random_state, df)
pickle.dump(Y_tta, open(f"{outdir}/{m_name}_tta.pkl", "wb"))
pickle.dump(Y_tta_hardvote, open(f"{outdir}/{m_name}_tta_hardvote.pkl", "wb"))
# display(Y_pred)
# display(Y_hardvote)
del models
torch.cuda.empty_cache()
gc.collect()

C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\notebook\cleanlab_test\20210125\kaggle_upload_tf_efficientnet_b4_ns_cleanlab_noise_cutmix_fmix_n_over\model_seed_83_fold_0.ckpt is loaded
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\notebook\cleanlab_test\20210125\kaggle_upload_tf_efficientnet_b4_ns_cleanlab_noise_cutmix_fmix_n_over\model_seed_83_fold_1.ckpt is loaded
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\notebook\cleanlab_test\20210125\kaggle_upload_tf_efficientnet_b4_ns_cleanlab_noise_cutmix_fmix_n_over\model_seed_83_fold_2.ckpt is loaded
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\notebook\cleanlab_test\20210125\kaggle_upload_tf_efficientnet_b4_ns_cleanlab_noise_cutmix_fmix_n_over\model_seed_83_fold_3.ckpt is loaded
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\notebook\cleanlab_test\20210125\kaggle_upload_tf_efficientnet_b4_ns_cleanlab_noise_cutmix_fmix_n_o


oof: 0.8881
oof_hardvote: 0.8871


28

In [20]:
# microsoftvisionresnet50
# oof -.---
m_dir = r"C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\kaggle_datasets_dl\cassava_microsoftvisionresnet50"
cfg = GetYaml(f"{m_dir}/cfg.yaml")
mdl = sorted(glob.glob(f"{m_dir}/model_seed_*.ckpt"))
models = get_models(mdl, "CassavaLite4", CFG=cfg)

m_name = f"microsoftvisionresnet50"
models = [(models, n_tta, cfg.height)]
random_state = cfg.seeds[0]
Y_tta, Y_tta_hardvote = check_train_oof(models, random_state, df)
pickle.dump(Y_tta, open(f"{outdir}/{m_name}_tta.pkl", "wb"))
pickle.dump(Y_tta_hardvote, open(f"{outdir}/{m_name}_tta_hardvote.pkl", "wb"))
# display(Y_pred)
# display(Y_hardvote)
del models
torch.cuda.empty_cache()
gc.collect()

Using default model initialization
Using default model initialization
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\kaggle_datasets_dl\cassava_microsoftvisionresnet50\model_seed_0_fold_0.ckpt is loaded
Using default model initialization
Using default model initialization
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\kaggle_datasets_dl\cassava_microsoftvisionresnet50\model_seed_0_fold_1.ckpt is loaded
Using default model initialization
Using default model initialization
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\kaggle_datasets_dl\cassava_microsoftvisionresnet50\model_seed_0_fold_2.ckpt is loaded
Using default model initialization
Using default model initialization
C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\kaggle_datasets_dl\cassava_microsoftvisionresnet50\model_seed_0_fold_3.ckpt is loaded
Using default model initialization
Using default model initialization
C:\Users\81908\jupyter_not


oof: 0.8866
oof_hardvote: 0.8827


28

In [ ]:
pickle.load(open(f"{outdir}/{m_name}_tta.pkl", "rb"))

In [ ]:
pickle.load(open(f"{outdir}/{m_name}_tta_hardvote.pkl", "rb"))

In [ ]:
if COMMIT:
    display(pd.DataFrame(test_preds_list[0], columns=name_mapping.values()))
    display(pd.DataFrame(test_preds_list[1], columns=name_mapping.values()))
    display(pd.DataFrame(test_preds_list[2], columns=name_mapping.values()))

### --- SiNpcw Module ---

In [ ]:
# tta operation
TTA_ROUND = 3

# image size
SIZE = 512

# class
NUM_CLASSES = 5

In [ ]:
def GetPath(pth):
    return os.path.join('../input/cassavapth/', pth)

modeldefs = [
    # Gr1.
    [
        # B4+FINE+UDA+TTA3 (LB900), CV: k0=8981, k1ep13=9030, k2ep13=8927, k3ep12=8983, k4ep14=8904
        { 'name' : 'efficientnet-b4', 'pth' : GetPath('22019613/22019613k0.pth') },
        { 'name' : 'efficientnet-b4', 'pth' : GetPath('22019613/22019613k1.pth') },
        { 'name' : 'efficientnet-b4', 'pth' : GetPath('22019613/22019613k2.pth') },
        { 'name' : 'efficientnet-b4', 'pth' : GetPath('22019613/22019613k3.pth') },
        { 'name' : 'efficientnet-b4', 'pth' : GetPath('22019613/22019613k4.pth') },
    ],
    # Gr2.
    [
        # resnest101e (LB896), CV: k0=9005, k1=9012, k2=8960, k3=8967, k4=8920
        { 'name' : 'timm-resnest101e', 'pth' : GetPath('22019632/22019632k0.pth') },
        { 'name' : 'timm-resnest101e', 'pth' : GetPath('22019632/22019632k1.pth') },
        { 'name' : 'timm-resnest101e', 'pth' : GetPath('22019632/22019632k2.pth') },
        { 'name' : 'timm-resnest101e', 'pth' : GetPath('22019632/22019632k3.pth') },
        { 'name' : 'timm-resnest101e', 'pth' : GetPath('22019632/22019632k4.pth') },
    ]
]

In [ ]:
def GetModel(name, param):
    num_classes = NUM_CLASSES
    if name in [ 'resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'resnext50', 'resnext101', 'wide_resnet50', 'wide_resnet101' ]:
        if name == 'resnext50' or name == 'resnext101':
            name = name + '_32x4d'
        elif name == 'wide_resnet50' or name == 'wide_resnet101':
            name = name + '_2'
        model = getattr(torchvision.models, name)(pretrained=None)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    elif name in [ 'senet154', 'se_resnet50', 'se_resnet101', 'se_resnet152', 'se_resnext50', 'se_resnext101', 'se_resnext50_32x4d', 'se_resnext101_32x4d' ]:
        if name == 'se_resnext50' or name == 'se_resnext101':
            name = name + '_32x4d'
        model = getattr(pretrainedmodels, name)(pretrained=None)
        model.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        model.last_linear = nn.Linear(model.last_linear.in_features, num_classes)
    elif name.startswith('efficientnet-b'):
        model = EfficientNet.from_name(name)
        model._fc = nn.Linear(model._fc.in_features, num_classes)
    elif name.startswith('timm-'):
        model = timm.create_model(model_name=name[len('timm-'):], num_classes=num_classes, in_chans=3, pretrained=False)
    else:
        raise NameError()
    state = torch.load(param, map_location=device)
    model.load_state_dict(state, strict=True)
    model.eval()
    print('model ({}) is loaded'.format(name))
    return model

In [ ]:
def GetAugment(size):
    return A.Compose([
        A.Resize(size, size),
        A.Normalize()
    ], p=1.0)

In [ ]:
def TTA(img, ops):
    # input: NxCxHxW
    if ops == 0:
        pass
    elif ops == 1:
        img = torch.flip(img, [-1])
    elif ops == 2:
        img = torch.flip(img, [-2])
    elif ops == 3:
        img = torch.flip(img, [-1, -2])
    elif ops == 4:
        img = torch.rot90(img, 1, [2, 3])
    elif ops == 5:
        img = torch.rot90(img, 3, [2, 3])
    else:
        pass
    return img

In [ ]:
def GetDataLoader(batch=8, num_workers=2):
    files = test_df.iloc[:, 0].values
    dataset = InferDataset(files, augops=GetAugment(SIZE))
    return torch.utils.data.DataLoader(
        dataset,
        batch_size=batch,
        shuffle=False,
        drop_last=False,
        num_workers=num_workers
    )

class InferDataset(Dataset):
    def __init__(self, files, augops):
        self.files = files
        self.augops = augops

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        img = cv2.imread(os.path.join(img_dir, self.files[idx]))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        out = self.augops(force_apply=False, image=img)['image']
        out = out.transpose(2, 0, 1)
        return torch.from_numpy(out)

loader = GetDataLoader(batch=8)

In [ ]:
modelgroups = [ ]
for i, groupdef in enumerate(modeldefs):
    print(f'--- Group{i} ---')
    models = [ ]
    for mdef in groupdef:
        mdl = GetModel(mdef['name'], mdef['pth']).to(device)
        models.append(mdl)
    modelgroups.append(models)

In [ ]:
with torch.no_grad():
    gnums = len(modelgroups)
    preds = [ [] for _ in range(gnums) ] * gnums
    actfn = nn.Softmax(dim=1)
    for _, itr in enumerate(loader):
        x = itr
        b = x.shape[0]
        x = x.to(device)
        y = torch.zeros([gnums, b, NUM_CLASSES], device=device)
        for tta in range(TTA_ROUND):
            xi = TTA(x, tta)
            for g, modelgroup in enumerate(modelgroups):
                ratio = 1.0 / len(modelgroup)
                for model in modelgroup:
                    y[g, :, :] = y[g, :, :] + actfn(model(xi)) * ratio
        if TTA_ROUND > 1:
            y = y / TTA_ROUND
        y = y.detach().cpu().numpy()
        for g in range(gnums):
            preds[g] += [ y[g, :, :] ]
for g in range(gnums):
    preds[g] = np.concatenate(preds[g], axis=0).reshape(-1, 5)
    test_preds_list.append(preds[g])

In [ ]:
if DEBUG:
    print(test_preds_list)

## ensemble

In [ ]:
# oofのcfmにもとづいた重み
# oof=0.9053605645651259
ens_weights = np.array(
    [
        [0.21396141, 0.20027513, 0.20551536, 0.19993096, 0.19595167],  # tf_efficientnet_b4_ns
        [0.20459705, 0.19993122, 0.20834644, 0.20013494, 0.20400593],  # byol_seresnext50_32x4d
        [0.1753689 , 0.1870916 , 0.18139876, 0.1981736 , 0.18853328],  # vit_b16_224_fold10
        [0.2122588 , 0.20382896, 0.20069204, 0.20110776, 0.20379398],  # 22019613_efficientnet-b4.pkl
        [0.19381385, 0.20887309, 0.20404739, 0.20065273, 0.20771513],  # 22019632_timm-resnest101e.pkl
    ]
)
test_preds_ensemble = np.zeros((test_df.shape[0], n_classes))
for i,pre in enumerate(test_preds_list):
    test_preds_ensemble += ens_weights[i] * pre
display(pd.DataFrame(test_preds_ensemble, columns=name_mapping.values()))

In [ ]:
test_df['label'] = test_preds_ensemble.argmax(1)
test_df.to_csv('submission.csv', index=False)
display(test_df)